In [1]:
import requests
import json
import time

In [ ]:
# Risinam gaismas ieslēgšanu - siltumnīcā ir uzstādītas nātrija lampas, ir zināma augiem nepieciešamā gaismas summa
# Cenšamies nesavākt vairāk gaismu, kā nepieciešams

global light_status_on
global nat_compare
global lightsum
global lightsum_lim
global off_tresh # Izslēgšanās limits
global off_counter # izslēgšanās laika skaitītājs
global on_off_lim # laika limits cik lampām jābūt ieslēgr\tām/izslēgtām
global on_tresh # atkārtotas ieslēgšanās limits
global on_counter

sleep=60
light_status_on=True # Mākslīgo gaismu -lampu ielēgšanās status, tiks caur I2C izvadīts uz releju gaismu darbināšanai
nat_compare=0 # Dabīgās gaismas līmenis
lightsum=0 #  kopējā gaismas summa, vēlāk nonuļļosim jaunas dienas sākumā
art_umol=0.0102 # Akumulētā gaismas summa pie 60sek darbības, jo uzstādīti 170umol/m2/s
natk=60 # sekundes minūtē, to izmantojam, jo var būt sensors, kur lasījumus nodod reizi 2 minūtēs -> 120sek
lightsum_lim=15 # Gaismas summas limits - sasniedzot 15 mol gaismas, lampas dedzināt nav nepieciešams
off_tresh=400 # Izslēgšanās limits, dabīgā gaisma umol/m2/s
off_counter=0 # izslēgšanās laika skaitītājs līdz brīdim kad var ieslēgt
on_off_lim=15 # Lampu izslēgšanas - ieslēgšanas laika limits - lampas pirms atkārtotas ieslēgšanas jaatdzesē 15 min, tā pat jauzsilda 15 min
on_tresh=300 # Atpakaļ ieslēgšanās limits, dabīgā gaisma umol/m2/s? pie kuras, ja nav savākta gaismas summa, lampas ieslēdzās
on_counter=0 # ieslēgšanās skaitītājs līdz brīdim kad var izslēgt

In [ ]:
# Norādam interesējošo sensoru url
natu='http://127.0.0.1:80/last/sensornum'
top_h='http://127.0.0.1:80/last/sensornum'
top_a='http://127.0.0.1:80/last/sensornum'
toplight=[top_h,top_a]

In [ ]:
# Sensora rādījuma pieprasījums
def get_data(url):
    resp=requests.get(url) #Saņemam atbildi
    data=resp.json() # izmantojam json atbildi
    num=data[0]['v'] # Paņemam nepieciešamo vērtību - tā kā šajā procesā intersē tikai sensora rādījums - to arī ņemam
    return num # Atgriežam vērtību

In [ ]:
def cumlightsum():
    global lightsum # Izmantosim mainīgo globālajā līmenī
    global nat_compare # dabīgās gaismas līmeni salīdzināšanai saglabājam globālajā mainīgajā
    for url in toplight: # Mākslīgajai gaismai izveidojām sarakstu, lai īsāks kods
        num=get_data(url) 
        if num>1:
            lightsum+=art_umol # Tā kā pie ieslēgtas pozīcijas (virs 1 volta) ir zināma gaismas summa, ko saražo, to pieskaitam
        else:
            continue
    nat_level=get_data(natu) # Paņemam dabīgās gaismas līmeni
    lightsum+=nat_level*natk # Tā kā sensora lasījumi jau ir molos, to reizinam ar sekundēm.
    nat_compare=nat_level*1000000 # Priekš salīdzināšanas vajag no moliem pāriet uz mikromoliem

In [ ]:
# Vienkāršākā daļa. šo vēlāk izmantošu pie laika loga kā vienu no izpildāmajām funkcijām, kur 
# definīcijas apgabals būs laika robežas no līdz, un gaismas summa nonuļļosies automātiski
def on_off_status():
    global lightsum
    global light_status_on
    global light_status_on
    global lightsum_lim
    global off_tresh
    global off_counter
    global on_off_lim
    global on_tresh
    global on_counter
    
    if lightsum>lightsum_lim and light_status_on==True:
        if on_counter>on_off_lim:
            light_status_on=False
            off_counter+=1
            on_counter=0
        else:
            on_counter+=1
            off_counter=0
    elif nat_compare>off_tresh and light_status_on==True:
        if on_counter>on_off_lim:
            light_status_on=False
            off_counter+=1
            on_counter=0
        else:
            on_counter+=1
            off_counter=0
    elif nat_compare<on_tresh and light_status_on==False and lightsum<lightsum_lim:
        if off_counter>on_off_lim:
            light_status_on=True
            on_counter+=1
            off_counter=0
        else:
            on_counter+=1
            off_counter=0
    elif light_status_on==False:
        on_counter=0
        off_counter+=1
    else:
        on_counter+=1
        off_counter=0

In [ ]:
# Palaižam mūžīgajā cilpā. Rakstot py failā uzrakstīšu citus pārtraukšanas noteikumus.
while True:
    try:
        cumlightsum()
        on_off_status()
        print("Gaismas summa ir {:0.2f}, status ir {}, on_count: {}, off_count:{}".format(lightsum,light_status_on, on_counter, off_counter))
        time.sleep(sleep)
    except:
        print("Kaut kas nogājis greizi, vajadzētu aizdomāties kas ne tā!")
        break